<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/credit_card_transactions_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 18th March, 2022
#               Holi
# Data source: https://ibm.ent.box.com/v/tabformer-data/folder/130747715605
# Data source simulator: https://fraud-detection-handbook.github.io/fraud-detection-handbook/Foreword.html
# Objective: Discovering community or collaboration among
#            credit-card fraudsters.

## Generate user/merchant nodes

### 1.0 Call libraries

In [1]:
# 1.0
import pandas as pd
import numpy as np
import os

In [2]:
# 1.1
pd.__version__  # 1.3.5

'1.3.5'

In [3]:
# 1.2 Display cell outputs from multiple commands
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 2.0 Mount google drive

In [4]:
# 2.0 Mount to ccma@fsm.ac.in
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
# 2.1 Check credit card data files
!ls /gdrive/MyDrive/credit_card_transactions/

card_transaction_v1.csv  creditCard_edges.csv	   creditCard_users.csv
card_transaction_v1.zip  creditCard_merchants.csv  sample_2002.csv


In [6]:
# 2.2 Change current directory
path = "/gdrive/MyDrive/credit_card_transactions/"
os.chdir(path)
os.listdir()

['.ipynb_checkpoints',
 'card_transaction_v1.csv',
 'card_transaction_v1.zip',
 'creditCard_users.csv',
 'creditCard_merchants.csv',
 'creditCard_edges.csv',
 'sample_2002.csv']

column names:  
User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?   
Sample data:  
0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,,No
0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,,No



### 3.0 Read full data and store a sample

In [8]:
# 3.0 Read data. File size is > 2gb
#     Takes around 
#     User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
#     MCC : Merchant Category code:  A four-digit number that describes a merchant's primary business activities. 
#     Feel free to include more fields (city, time of transaction etc) and analyse:
#
%%time
df = pd.read_csv("card_transaction_v1.csv",
                 usecols = ["User", "Year", "Merchant Name", "Merchant City", "Amount", "MCC", "Is Fraud?"])

CPU times: user 26.9 s, sys: 4.64 s, total: 31.6 s
Wall time: 44 s


In [9]:
# 3.1 Our data:
df.head()
df.tail()
df.shape   # (24386900, 6)

,User,Year,Amount,Merchant Name,Merchant City,MCC,Is Fraud?
0,0,2002,$134.09,3527213246127876953,La Verne,5300,No
1,0,2002,$38.48,-727612092139916043,Monterey Park,5411,No
2,0,2002,$120.34,-727612092139916043,Monterey Park,5411,No
3,0,2002,$128.95,3414527459579106770,Monterey Park,5651,No
4,0,2002,$104.71,5817218446178736267,La Verne,5912,No


,User,Year,Amount,Merchant Name,Merchant City,MCC,Is Fraud?
24386895,1999,2020,$-54.00,-5162038175624867091,Merrimack,5541,No
24386896,1999,2020,$54.00,-5162038175624867091,Merrimack,5541,No
24386897,1999,2020,$59.15,2500998799892805156,Merrimack,4121,No
24386898,1999,2020,$43.12,2500998799892805156,Merrimack,4121,No
24386899,1999,2020,$45.13,4751695835751691036,Merrimack,5814,No


(24386900, 7)

In [10]:
# 3.2 Attempt should be made to
#     reduce data size:

df.dtypes

User              int64
Year              int64
Amount           object
Merchant Name     int64
Merchant City    object
MCC               int64
Is Fraud?        object
dtype: object

In [11]:
# 3.3 Is data balanced?

df['Is Fraud?'].value_counts()
print("\n")
df['Is Fraud?'].value_counts(normalize = True)  # 99.78% vs 0.12%

No     24357143
Yes       29757
Name: Is Fraud?, dtype: int64

No     0.99878
Yes    0.00122
Name: Is Fraud?, dtype: float64

In [12]:
# 3.4 Remove '$' sign from 'Amount':
#     Takes time 15 secs:
%%time
df['Amount']=df['Amount'].str.replace("$", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


CPU times: user 11.7 s, sys: 996 ms, total: 12.7 s
Wall time: 13.5 s


In [13]:
# 3.5 Convert 'Amount' to numeric:
df['Amount'] = pd.to_numeric(df['Amount'])

In [15]:
# 3.6 Data shape
df.shape   # rows: 243,86,900; cols: 6)

(24386900, 7)

#### Store a sample of data

In [16]:
# 4.0 As RAM is limited, we will work
#     with a sample for the year 2002:

dfsample = df.loc[df['Year'] == 2002, :]

In [17]:
# 4.1 Sample size: 
dfsample.shape   # (350732, 6)

(350732, 7)

In [18]:
# 4.2 How many incidents of frauds?
dfsample['Is Fraud?'].value_counts()   # Yes: 139, No: 350593

No     350593
Yes       139
Name: Is Fraud?, dtype: int64

In [19]:
# 4.3 Should you like to save this sample
#     for latter quick reading?

dfsample.to_csv("sample_2002.csv", index = False)

### 4.0 Read sample of data

In [7]:
%%time
df = pd.read_csv(path+"sample_2002.csv")                 

CPU times: user 239 ms, sys: 73.8 ms, total: 312 ms
Wall time: 341 ms


### 5.0 Perform processing on data

In [8]:
# 4.4 We work with a copy of data
#     If, we make mistakes, we come back here:

df_sample = df.copy()

In [9]:
# 4.5 And our data:
df_sample.head()

,User,Year,Amount,Merchant Name,Merchant City,MCC,Is Fraud?
0,0,2002,134.09,3527213246127876953,La Verne,5300,No
1,0,2002,38.48,-727612092139916043,Monterey Park,5411,No
2,0,2002,120.34,-727612092139916043,Monterey Park,5411,No
3,0,2002,128.95,3414527459579106770,Monterey Park,5651,No
4,0,2002,104.71,5817218446178736267,La Verne,5912,No


### Change user IDs
> 1.0 Know unique user names and transform them to short names  
> 2.0 Prepare a dictionary of user-ids and proposed Ids  
> 3.0 Make changes to our dataset using the dictionary  

In [60]:
# 5.0 To distinguish userids on network graph,
#     we will prefix userids with 'u' and also assign
#      them a short name so that they fit within nodes 
#       when displayed on graph:

#    First get unique user names
u_user = df_sample['User'].unique()

In [ ]:
# 5.1
u_user.sort()
u_user
print("\n")  
len(u_user)  # 426

In [27]:
# 5.2 Code due to Alakshendra
u_code = ["u" + str(i) for i in range(len(u_user)) ]
map_dict = dict(zip(u_user, u_code))
map_dict

In [ ]:
# 5.2 Get alternate names by prefixing userids with 'u'

# 5.2.1 Transform sorted-user array to a dataframe; Get its index as a column; Rename new column as 'alt_name'
#       So, alternate names are sequenial: 0,1,2,3.. and final names will be: u1,u2,u3...
dx = pd.DataFrame(u_user, columns = ["u_user"]).reset_index().rename(columns = {'index' : 'alt_name' })

# 5.2.2 Transform the new column to string
dx['alt_name'] = dx['alt_name'].apply(str)

# 5.2.3 Create a new column 'a' with a constant value:
dx['a'] = "u"

# 5.2.4 Concatenate 'a' column and 'alt_name' column:
dx['alt_name'] = dx['a'].str.cat(dx['alt_name']) 

# 5.2.5 Drop 'a'
_=dx.pop('a')

# 5.2.6 Check:
dx.head()

In [ ]:
# 5.3 Prepare a dictionary having
#     old names (key) and new names (value) 
#     for further transformation:

# https://stackoverflow.com/a/20250947/3282777
map_dict = dict(zip(dx['u_user'], dx['alt_name']))
map_dict

In [28]:
# 5.4 Use 'replace' method to replace values in column 'User':

df_sample['User'] = df_sample['User'].replace(map_dict, inplace= False)

In [ ]:
# 5.5 Check:
df_sample.head()

### Change merchant IDs
Same steps for changing long merchantIds with short IDs. We prefix these Ids with 'm'

In [30]:
# 6.0 Unique merchant ids
mn = df_sample['Merchant Name'].unique()

In [31]:
# 6.1 Sort them in place
mn.sort()

In [ ]:
# Code due to Alakshendra
merchant_code = ["m" + str(i) for i in range(len(mn)) ]
map_dict = dict(zip(mn, merchant_code))
map_dict

In [ ]:
# 6.2 Prepare a dataframe of existing and alternate names:
dx = pd.DataFrame(mn, columns = ["mt"]).reset_index().rename(columns = {'index' : 'alt_name' })
dx['alt_name'] = dx['alt_name'].apply(str)
dx['a'] = "m"
dx['alt_name'] = dx['a'].str.cat(dx['alt_name']) 
_=dx.pop('a')
dx.head()

In [ ]:
# 6.3 Create a dictionary for transformation:
# https://stackoverflow.com/a/20250947/3282777
map_dict = dict(zip(dx['mt'], dx['alt_name']))
map_dict

In [33]:
# 6.4 Transform values in 'Merchant Name' using map_dict:
#     Takes time 4 minutes
%%time
df_sample['Merchant Name'] = df_sample['Merchant Name'].replace(map_dict, inplace= False)

CPU times: user 13.8 s, sys: 2min 18s, total: 2min 32s
Wall time: 2min 34s


In [ ]:
# 6.5 And check
df_sample.head()

### User Nodes   


#### Fields

>This node must have two <i>must</i> fields: Id and Label. Besides these two, to distinguish Users from Merchants, we also have a 'Cat' field. The 'Cat' field has two values: *Member* for Users and *Institution* for Merchants.  

> Besides, these three common field names, other fields may also be there and may carry any names.


> If a user has committed fraud, even once, we mark him with propensity to commit frauds. A column 'suspect' is added to records this propensity.  

In [ ]:
## 7.0 Group by user to get user node charteristics:
# StackOverflow:  https://stackoverflow.com/a/68726106/3282777
grpd_user = df_sample.groupby(['User'])
user_nodes = grpd_user.agg({'Amount' : [('u_min','min'),('u_max','max'),('u_mean','mean'),('u_std',np.std)] }).reset_index()
user_nodes = user_nodes.round(decimals = 2)
user_nodes.head()

In [36]:
# 7.1 Which of the users have committed fraud
users_suspect = df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'User' ].unique()
len(users_suspect)  # 25

25

In [37]:
# 7.2 Add a column 'suspect' with default value of 'No'
user_nodes['suspect'] = "No" 

# 7.3 To distinguish user nodes from merchant nodes on the graph
#     we add a 'Cat' column.

user_nodes['Cat'] = 'Member'

In [38]:
# 7.4 Even if a user committed fraud once, we set value
#     in 'suspect' as 'Yes' for every transaction:

for i in users_suspect:
  user_nodes.loc[user_nodes['User'] == i, 'suspect' ] = "Yes"  

In [ ]:
# 7.5 So how many of them?

user_nodes['suspect'].value_counts()  # No: 401, 'Yes: 25
user_nodes.head()

In [40]:
# 7.6 Create a 'Label' column
#     that simply records userids.
#     Or, rather a copy of 'User' feature
#     'Label' column is a MUST in a graph:

user_nodes['Label'] = user_nodes['User']

# 7.7 Also a graph should have an 'Id' column
#     We rename 'User' field as 'Id'

user_nodes = user_nodes.rename(columns = {'User': "Id"})

In [ ]:
# 7.8 our user nodes data:
user_nodes.head()

In [42]:
# 7.9 We also create a column 'suspect_n' that records
#     1 for 'Yes' fraud and 0 for 'No' fraud
#     It is simply a numeric transformation of 'suspect' field:
#     We do this as data manipulation capabilities within gephi
#     are limited:

user_nodes['suspect_n'] = user_nodes['suspect']
user_nodes['suspect_n'] = user_nodes['suspect_n'].map({"Yes":1 , "No" : 0})

In [ ]:
# 7.11
user_nodes.head()

In [44]:
! rm  /gdrive/MyDrive/credit_card_transactions/creditCard_users.csv

In [45]:
# 7.12 Save usernodes to gdrive with semicolon separator:
#     Header is stored as:
#       Id;Amount;Amount;Amount;Amount;suspect;Cat;Label;suspect_n
#       ;u_min;u_max;u_mean;u_std;;;;
#     Change this manually to:
#       Id;u_min;u_max;u_mean;u_std;suspect;Cat;Label;suspect_n

user_nodes.to_csv("creditCard_users.csv",
                  index = False,
                  sep = ";"
                  )

### Merchant nodes

In [ ]:
df_sample.head()

In [ ]:
## 8.0 Group by merchant and get merchant charteristics:
grpd_merchant = df_sample.groupby(['Merchant Name'])
merchant_nodes = grpd_merchant.agg({'Amount' : [('m_min','min'),('m_max','max'),('m_mean','mean'),('m_std', 'std')]}).reset_index()
merchant_nodes = merchant_nodes.round(2)
merchant_nodes.head()
merchant_nodes.shape   # (10853, 2)

In [ ]:
## 8.01 Group by merchant and merchant city:
grpd_merchant1 = df_sample.groupby(['Merchant Name','Merchant City'])
merchant_nodes1 = grpd_merchant1['Amount'].mean().reset_index()
merchant_nodes1 = merchant_nodes1.round(2)
merchant_nodes1.head()
merchant_nodes1.shape    # (25984, 3). It appears a Merchant operates in more than one City

In [ ]:
## 8.01 Group by merchant and MCC:
grpd_merchant2 = df_sample.groupby(['Merchant Name','MCC'])
merchant_nodes2 = grpd_merchant2['Amount'].mean().reset_index()
merchant_nodes2 = merchant_nodes2.round(2)
merchant_nodes2.head()
merchant_nodes2.shape    # (10875, 3). It appears a Merchant has more than one line of business
                         #  or two merchants have the same name. 

In [ ]:
# 8.1 With which one of the merchants fraud comitted:
df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'Merchant Name' ].unique()
merchant_suspect = df_sample.loc[df_sample["Is Fraud?"] == "Yes", 'Merchant Name' ].unique()
len(merchant_suspect)

In [51]:
# 8.2 We record this also in 'victim':
merchant_nodes['victim'] = "No" 

# 8.3 Merchant 'Cat' we designate as 'Institution'
merchant_nodes['Cat'] = 'Institution'

In [52]:
# 8.3 Record which all merchants have propensity to become victims:

for i in merchant_suspect:
  merchant_nodes.loc[merchant_nodes['Merchant Name'] == i, 'victim' ] = "Yes"
  

In [ ]:
# 8.4
merchant_nodes['victim'].value_counts()  # 10777, 76

In [ ]:
# 8.5 Label and Id columns of Merchant nodes:

merchant_nodes['Label'] = merchant_nodes['Merchant Name']
merchant_nodes = merchant_nodes.rename(columns = {'Merchant Name': "Id"})
merchant_nodes.head()
merchant_nodes.shape

In [55]:
# 8.5 We have a numeric field victim_n recording the
#     same information as 'victim':

merchant_nodes['victim_n'] = merchant_nodes['victim']
merchant_nodes['victim_n'] = merchant_nodes['victim_n'].map({"Yes":1 , "No" : 0})

In [ ]:
# 8.5.1
merchant_nodes.head()
merchant_nodes.shape   # (10853, 6)

In [ ]:
! rm  /gdrive/MyDrive/credit_card_transactions/creditCard_merchants.csv

In [ ]:
# 8.6 Save merchant_nodes information to a file:

merchant_nodes.to_csv("creditCard_merchants.csv",
                      index = False,
                      sep = ";")

### Edges

Edges must have three fields: *Source* , *Target* and *Type*. *Type* field records if an edge is directed or undirected. Besides these three, it may have other fields also.

In [ ]:
# 9.0 Group by User and Merchant Name:
#     We also wish to record the number of interactions between
#     customer and merchants. These will be edge weights:

edges = df_sample.groupby([df_sample['User'], df_sample['Merchant Name']]).size()

In [ ]:
# 9.1 The size column has a name '0'. We need to rename it:

edges = df_sample[['User', 'Merchant Name']].groupby(['User', 'Merchant Name']).size().reset_index()
edges = edges.rename(columns = { 0 : "weight"})
edges.head()

In [ ]:
# 9.2 Rename two other columns appropriately:

edges = edges.rename(columns = {'User': 'Source', 'Merchant Name': 'Target'})

In [ ]:
# 9.3 Our edges are Undirected:

edges['Type']= 'Undirected'
edges.head()

In [ ]:
! rm  /gdrive/MyDrive/credit_card_transactions/creditCard_edges.csv

In [ ]:
# 9.4 Finally save edges information to a file:

edges.to_csv("creditCard_edges.csv",
             index = False,
             sep = ";"
             )

### Check files


In [ ]:
! ls -la !ls /gdrive/MyDrive/credit_card_transactions/